In [1]:
import numpy as np
import qiskit as qk
import matplotlib.pyplot as plt

from qiskit import Aer
from tqdm.notebook import tqdm
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split

import sys
sys.path.insert(0, '../../src/')
from neuralnetwork import *
from analysis import *

%matplotlib notebook
#%matplotlib inline
%load_ext autoreload
%autoreload 2

### Digits data

In [2]:
digits = load_digits()
zero_idx = (digits.target == 0)
one_idx = (digits.target == 1)
two_idx = (digits.target == 2)

In [3]:
zeros = digits.data[zero_idx]
ones =  digits.data[one_idx]
twos = digits.data[two_idx]
#threes = digits.data[three_idx]

x = np.concatenate((zeros, ones, twos))
x = x - np.mean(x)
x = x/np.max(np.abs(x))

y = np.zeros((x.shape[0], 3))
y[:len(zeros), 0] = 1
y[len(zeros):(len(zeros)+len(ones)), 1] = 1
y[(len(zeros)+len(ones)):, 2] = 1
#y[(len(zeros)+len(ones) + len(twos)):, 3] = 1
print(y)

[[1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 ...
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]]


In [4]:
np.random.seed(42)
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=100)
print(x_train.shape)

(100, 64)


### Network

In [5]:
np.random.seed(42)
optimizer = Adam(lr=1e-1)
backend = Aer.get_backend('qasm_simulator')

layer1 = Dense(n_features=64, 
               n_targets=4, 
               scale = 2*np.pi, 
               activation = Identity())

layer2 = QLayer(n_qubits=4, 
                n_features=4, 
                n_targets=4, 
                encoder=Encoder(), 
                ansatz=Ansatz(), 
                sampler = Parity(),
                reps=2, 
                scale=2*np.pi, 
                backend=backend, 
                shots=10000)

layer3 = QLayer(n_qubits=4, 
                n_features=4, 
                n_targets=3, 
                encoder=Encoder(), 
                ansatz=Ansatz(), 
                sampler = Parity(),
                reps=2, 
                scale=1, 
                backend=backend, 
                shots=10000)

layers =[layer1, layer2, layer3]
network1 = NeuralNetwork(layers, optimizer)

### Training

In [ ]:
network1.train(x_train, y_train, epochs=100, verbose=True)
saver(network1, data_path("digits_hybrid_layers_3"))

  0%|          | 0/100 [00:00<?, ?it/s]

epoch: 0, loss: 0.2700916664333333


In [10]:
np.random.seed(42)
network2 = sequential_dnn(dim=[64, 4, 4, 3], lr=0.1)

In [11]:
network2.train(x_train, y_train, epochs=100, verbose=True)

  0%|          | 0/100 [00:00<?, ?it/s]

epoch: 0, loss: 0.37241333163555157
epoch: 1, loss: 0.32650439400317066
epoch: 2, loss: 0.2908751550939861
epoch: 3, loss: 0.26978843214073506
epoch: 4, loss: 0.2538358824088277
epoch: 5, loss: 0.24057315912469301
epoch: 6, loss: 0.2298832325398529
epoch: 7, loss: 0.22178879179664293
epoch: 8, loss: 0.21571441753373904
epoch: 9, loss: 0.21122334142640356
epoch: 10, loss: 0.20824004611955535
epoch: 11, loss: 0.20637317887815376
epoch: 12, loss: 0.2044346576011082
epoch: 13, loss: 0.20086124902268043
epoch: 14, loss: 0.1940848561195667
epoch: 15, loss: 0.1861402259515153
epoch: 16, loss: 0.18011905052276916
epoch: 17, loss: 0.17504055039645144
epoch: 18, loss: 0.1699537192254483
epoch: 19, loss: 0.16430975373672735
epoch: 20, loss: 0.15789826619730887
epoch: 21, loss: 0.1507910436393874
epoch: 22, loss: 0.1428924849869262
epoch: 23, loss: 0.1340404501419498
epoch: 24, loss: 0.1253583420181962
epoch: 25, loss: 0.11747388728810859
epoch: 26, loss: 0.1097264322629123
epoch: 27, loss: 0.1016

In [13]:
y_pred = np.round(network1.predict(x_train))
print(np.mean(y_pred == y_train))

1.0


In [14]:
y_pred = np.round(network2.predict(x_train))
print(np.mean(y_pred == y_train))

1.0
